# Verseagility - Score Model
- A script to batch-score your endpoint after the deployment.
- Set your endpoint and keys in the function below
- Read your test data set which has a "label" column for the ground truth and a "text" column with the document to be scored

In [ ]:
# Import packages
import requests
import pandas as pd
import json
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
# Change this to your format respectively
df_test = pd.read_csv("file.csv", sep=";", encoding="utf-8")

In [ ]:
# Endpoint settings
endpoint = '[YOUR URL GOES HERE]'
region = '[YOUR REGION GOES HERE]'
key = '[YOUR KEY GOES HERE]'

In [ ]:
# Model scoring function
def score_model(df, endpoint, region, key):
    # URL for the web service
    scoring_uri = f'http://{endpoint}.{region}.azurecontainer.io/score'
    # Set the content type
    headers = {'Content-Type': 'application/json'}
    # If authentication is enabled, set the authorization header
    headers['Authorization'] = f'Bearer {key}'
    scores = []
    # Iterate
    for index, row in df.iterrows():
        data = [{
            "subject": "",
            "body": str(row['text'].replace("'", "").replace('"', ''))
        }]
        # Convert to JSON string
        input_data = json.dumps(data)
        # Make the request and display the response
        resp = requests.post(scoring_uri, input_data, headers=headers)
        try:
            pred = json.loads(resp.text)[0]['result'][0]['category']
        except:
            pred = "None"
        scores.append(pred)
        print(f'[INFO] - SCORING {str(index+1)}/{len(df)} -> "{row["label"]}" predicted as "{pred}"')
    return scores

In [ ]:
# Initiate the scoring
scores = score_model(df_test, endpoint, region, key)

In [ ]:
# Get your classification reports after scoring
print(classification_report(df_test['label'], scores))
print(confusion_matrix(df_test['label'], scores))
print(f"Accuracy: {accuracy_score(df_test['label'], scores)}")

In [ ]:
# Write output to file
scores.to_csv("scoring.csv", sep=";")